In [ ]:
import os
import sys
import subprocess
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession

import findspark
findspark.init()
#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [3]:
# Create Spark session
spark = SparkSession.builder.appName("HMP Spark Loader").getOrCreate()
spark.catalog.clearCache()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/05 02:44:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/05 02:44:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
import os

def list_available_activities(base_path="HMP_Dataset"):
    return sorted([f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))])

# Example usage
available_activities = list_available_activities()
print("Available activities:")
for act in available_activities:
    print("-", act)


FileNotFoundError: [Errno 2] No such file or directory: 'HMP_Dataset'

In [5]:
def load_hmp_from_repo(base_path="HMP_Dataset", selected_activities=None):
    rows = []
    for activity in os.listdir(base_path):
        if selected_activities and activity not in selected_activities:
            continue
        activity_path = os.path.join(base_path, activity)
        if not os.path.isdir(activity_path):
            continue
        for fname in os.listdir(activity_path):
            fpath = os.path.join(activity_path, fname)
            with open(fpath, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 3:
                        x, y, z = map(float, parts)
                        rows.append((activity, x, y, z))  # ✅ tuple (not keyword args!)
    return rows
